# STAC EuroSAT

In this demo we generate STAC metadata for the [EuroSAT](https://github.com/phelber/EuroSAT) dataset.

In [31]:
# data download

import requests
import os 
from pathlib import Path
import zipfile

url = 'https://madm.dfki.de/files/sentinel/EuroSATallBands.zip'
path = Path('data')

if not os.path.exists(path / 'ds'):
	r = requests.get(url, allow_redirects=True)
	open('data/EuroSATallBands.zip', 'wb').write(r.content)
	with zipfile.ZipFile(path / 'EuroSATallBands.zip', 'r') as zip_ref:
		zip_ref.extractall(path)

The EuroSAT dataset consists of 2700 Sentinel 2 images with one label per image for scene classification. There are 10 different categories in total. We use 100 samples for fast prototyping.

In [32]:
import pandas 
from glob import glob 
from random import sample

images = glob(str(path) + '/ds/**/*.tif', recursive=True)
images = sample(images, 100)
labels = [x.split('/')[-1].split('_')[0] for x in images]
cats = sorted(os.listdir(path / 'ds/images/remote_sensing/otherDatasets/sentinel_2/tif'))
ixs = [cats.index(x) for x in labels]

df = pandas.DataFrame({'image': images, 'label': labels, 'ix': ixs})
df

,image,label,ix
0,data/ds/images/remote_sensing/otherDatasets/se...,Pasture,5
1,data/ds/images/remote_sensing/otherDatasets/se...,Highway,3
2,data/ds/images/remote_sensing/otherDatasets/se...,PermanentCrop,6
3,data/ds/images/remote_sensing/otherDatasets/se...,Residential,7
4,data/ds/images/remote_sensing/otherDatasets/se...,Residential,7
...,...,...,...
95,data/ds/images/remote_sensing/otherDatasets/se...,Forest,1
96,data/ds/images/remote_sensing/otherDatasets/se...,Residential,7
97,data/ds/images/remote_sensing/otherDatasets/se...,SeaLake,9
98,data/ds/images/remote_sensing/otherDatasets/se...,River,8


In [33]:
df.ix.unique()

array([5, 3, 6, 7, 2, 1, 9, 4, 0, 8])

We start by generating STAC metadata following the core STAC specification. 

- We generate a STAC item for every image in the datasets
- a STAC collection to represent the images collection
- a STAC catalog to represent the final dataset (which will include also the annotations).

https://pystac.readthedocs.io/en/stable/

In [34]:
import pystac
from datetime import datetime
import rasterio as rio
import uuid
from shapely.geometry import GeometryCollection, Polygon, box, shape, mapping
from tqdm import tqdm

In [35]:
# create empty catalog

eurosat = pystac.Catalog(id="eurosat", description="EuroSAT dataset")
eurosat

ID: eurosat
Description: EuroSAT dataset
Rel: root
Target:
Media Type: application/json


In [36]:
# create collection

# # spatial extent (should compute from images)
sp_extent = pystac.SpatialExtent([None,None,None,None])

# temporal extentn (should compute from images or given by authors)
from_date = datetime.strptime('2015-10-22', '%Y-%m-%d') # unknown
to_date = datetime.strptime('2019-10-22', '%Y-%m-%d') # unknown
tmp_extent = pystac.TemporalExtent([(from_date, to_date)])

extent = pystac.Extent(sp_extent, tmp_extent)

sentinel = pystac.Collection(id='sentinel2', description = 'EuroSAT Sentinel 2 dataset', extent = extent)
eurosat.add_child(sentinel)

eurosat

ID: eurosat
Description: EuroSAT dataset
ID: sentinel2
Description: EuroSAT Sentinel 2 dataset
Rel: root
Target:
Media Type: application/json
Rel: parent
Target:
Media Type: application/json
Rel: root


In [37]:
# creating items

dst_path = path / 'eurosat'
def create_item(image):
    params = {}
    params['id'] = image.split('/')[-1].split('.')[0] # use original name
    params['datetime'] = from_date # unknown
    params['properties'] = {}
    with rio.open(image) as src:
        params['bbox'] = list(src.bounds)
        params['geometry'] = mapping(box(*params['bbox']))
        i = pystac.Item(**params)
        image_dst_path = dst_path / f"{params['id']}.tif"
        for band in src.indexes:
            image_dst_path = dst_path / f"{params['id']}_B{band}.tif"
            out_meta = src.meta.copy()
            out_meta.update({"count": 1})
            with rio.open(image_dst_path, "w", **out_meta) as dest:
                dest.write(src.read(band), 1)
            i.add_asset(key=f'B{band}', asset=pystac.Asset(href=str(image_dst_path), title='Geotiff', media_type=pystac.MediaType.GEOTIFF))
    return i

In [38]:
import multiprocessing
from concurrent.futures import ProcessPoolExecutor

num_cores = multiprocessing.cpu_count()
with ProcessPoolExecutor(max_workers=num_cores) as pool:
    with tqdm(total=len(images)) as progress:
        futures = []
        for image in df.image:
            future = pool.submit(create_item, image) 
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)
        items = []
        for future in futures:
            result = future.result()
            items.append(result)
            
for item in tqdm(items):
  sentinel.add_item(item)

100%|██████████| 100/100 [00:00<00:00, 39900.15it/s]


In [39]:
# reset spatial extent

bounds = [list(GeometryCollection([shape(s.geometry) for s in eurosat.get_all_items()]).bounds)]
sentinel.extent.spatial = pystac.SpatialExtent(bounds)

In [40]:
eurosat.normalize_hrefs('eurosat-stac')

In [41]:
eurosat.validate_all()

In [42]:
eurosat.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

We have created a STAC Catalog for our dataset !

In [43]:
eurosat = pystac.Catalog.from_file('eurosat-stac/catalog.json')
eurosat

ID: eurosat
Description: EuroSAT dataset
type: Catalog
ID: sentinel2
Description: EuroSAT Sentinel 2 dataset
type: Collection
stac_extensions: []
ID: Pasture_354
"Bounding Box: [557139.766364, 5882832.541223793, 557778.4111188718, 5883473.46161]"
Datetime: 2015-10-22 00:00:00+00:00
datetime: 2015-10-22T00:00:00Z
